### Pinecone Vector DB



In [ ]:
## Create your index and apikey from here https://www.pinecone.io/
import os
api_key=os.getenv("PINECONE_API_KEY")
from pinecone import Pinecone
pc=Pinecone(api_key=api_key)


In [2]:
# !pip install -qU langchain langchain-pinecone langchain-openai

In [2]:
from pinecone import Pinecone
pc=Pinecone(api_key=api_key)

In [3]:
from langchain_openai import OpenAIEmbeddings
from app.utils.model_loader import ModelLoader
model_loader = ModelLoader(model_provider="huggingface")
embeddings_model = model_loader.load_llm()
# embeddings = OpenAIEmbeddings(model="text-embedding-3-small",dimensions=1024,api_key="openai_api_key")

Loading config....
LLM loading...
Loading model from provider: 
Loading model from huggingface:


c:\code\Bajaj HackRx\Rag_app\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from pinecone import ServerlessSpec

index_name = "rag-project"  # change if desired

# if not pc.has_index(index_name):
#     pc.create_index(
#         name=index_name,
#         dimension=1024,
#         metric="cosine",
#         spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    # )

index = pc.Index(index_name)

In [9]:
query = "National Parivar Mediclaim Plus Policy"
query_embed = embeddings_model.embed_query(query)
index.query(
    vector=query_embed,
    top_k=2,
    namespace="hackrx-index2025-09-09-18-56"
)

{'matches': [{'id': 'db74f8fa-1f92-4395-8832-474ffe40781a',
              'score': 0.790277302,
              'values': []},
             {'id': 'a1da8bc6-657e-4ef0-aea4-00fd2974f047',
              'score': 0.789620757,
              'values': []}],
 'namespace': 'hackrx-index2025-09-09-18-56',
 'usage': {'read_units': 1}}

In [17]:
index.__getattr__

<bound method PluginAware.__getattr__ of <pinecone.db_data.index.Index object at 0x0000015D4F754590>>

In [10]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings_model, namespace="hackrx-index2025-09-09-18-56")

In [11]:
vector_store

In [14]:
### Query Directly
results = vector_store.similarity_search(
    "under the enactments specified under Schedule of Section 56(1) of the said Act,",
    k=2
    )
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

In [15]:
results

[]

In [23]:
results = vector_store.similarity_search_with_score(
    "National Parivar Mediclaim Plus Policy", k=1
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

In [16]:
### Retriever
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 1, "score_threshold": 0.4},
)
result = retriever.invoke("National Parivar Mediclaim Plus Policy")

No relevant docs were retrieved using the relevance score threshold 0.4


In [ ]:
type(result)

IndexError: list index out of range